In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**#unzip files**

In [1]:
!unzip '/kaggle/input/genre-classification-20'

unzip:  cannot find or open /kaggle/input/genre-classification-20, /kaggle/input/genre-classification-20.zip or /kaggle/input/genre-classification-20.ZIP.


**#imports**

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import transformers
from tqdm import tqdm
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

**#data loading**

In [4]:
df=pd.read_csv('/kaggle/input/genre-classification-20/Genre Classification Dataset/train_data.txt',sep=':::',header=None, names=['ID','TITLE','GENRE','DESCRIPTION'])

<ipython-input-4-0082dd271044>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df=pd.read_csv('/kaggle/input/genre-classification-20/Genre Classification Dataset/train_data.txt',sep=':::',header=None, names=['ID','TITLE','GENRE','DESCRIPTION'])


In [5]:
df.head()

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [6]:
df['GENRE'].unique()

array([' drama ', ' thriller ', ' adult ', ' documentary ', ' comedy ',
       ' crime ', ' reality-tv ', ' horror ', ' sport ', ' animation ',
       ' action ', ' fantasy ', ' short ', ' sci-fi ', ' music ',
       ' adventure ', ' talk-show ', ' western ', ' family ', ' mystery ',
       ' history ', ' news ', ' biography ', ' romance ', ' game-show ',
       ' musical ', ' war '], dtype=object)

In [7]:
df.isnull().sum()

ID             0
TITLE          0
GENRE          0
DESCRIPTION    0
dtype: int64

**#splitting data into training and validation**

In [8]:
x=df.drop(['ID','GENRE'],axis=1)
y=df['GENRE']

In [9]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

In [10]:
f"traing data size :{x_train.shape}"

'traing data size :(43371, 2)'

In [11]:
f"validation data size :{x_val.shape}"

'validation data size :(10843, 2)'

**#training data**

In [12]:
x_train.head()

,TITLE,DESCRIPTION
27179,Rivals (1972),"Scott Jacoby, as a boy with an unhealthy and ..."
19974,Kosava (1974),The story of two workers who returned from ab...
48283,In Winter (2017),In Winter is an independent feature emerging ...
37539,Maria Chapdelaine (1950),"At the beginning of the 20th century, in the ..."
43388,The Gift Of (2018),A delicious combo of romantic-comedy and soci...


In [13]:
y_train.head()

27179      drama 
19974      drama 
48283      drama 
37539      drama 
43388     comedy 
Name: GENRE, dtype: object

**#data preprocessing and cleaning**

In [14]:
'''
converting targeted variables into numbers using lable encoder
converting string into tokens using tokenizer
'''

'\nconverting targeted variables into numbers using lable encoder\nconverting string into tokens using tokenizer\n'

In [15]:
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_val=le.transform(y_val)

In [16]:
len(le.classes_)

27

In [17]:
d={}
for i in range(0,27):
  d[i]=le.inverse_transform([i])[0]
print(d)

{0: ' action ', 1: ' adult ', 2: ' adventure ', 3: ' animation ', 4: ' biography ', 5: ' comedy ', 6: ' crime ', 7: ' documentary ', 8: ' drama ', 9: ' family ', 10: ' fantasy ', 11: ' game-show ', 12: ' history ', 13: ' horror ', 14: ' music ', 15: ' musical ', 16: ' mystery ', 17: ' news ', 18: ' reality-tv ', 19: ' romance ', 20: ' sci-fi ', 21: ' short ', 22: ' sport ', 23: ' talk-show ', 24: ' thriller ', 25: ' war ', 26: ' western '}


In [18]:
def text_preprocessing(x,y,traing=None,validation=None,Testing=None):
  train_data=[]
  train_tragets=[]
  val_data=[]
  val_targets=[]
  test_data=[]
  test_targets=[]
  if traing==True:
    for i in tqdm(range(len(x))):
      review='TITLE :'+ str(x['TITLE'].iloc[i]) + ', DESCRIPTION :' + str(x['DESCRIPTION'].iloc[i])
      review=' '.join(review.split())
      train_data.append(review)
      train_tragets.append(y[i])
    return train_data,train_tragets
  if validation==True:
    for i in tqdm(range(len(x))):
      val_review='TITLE :'+ str(x['TITLE'].iloc[i]) + ', DESCRIPTION :' + str(x['DESCRIPTION'].iloc[i])
      val_review=' '.join(val_review.split())
      val_data.append(val_review)
      val_targets.append(y_val[i])
    return val_data,val_targets
  if Testing==True:
    for i in tqdm(range(len(x))):
      review='TITLE :'+ str(x['TITLE'].iloc[i]) + ', DESCRIPTION :' + str(x['DESCRIPTION'].iloc[i])
      review=' '.join(review.split())
      test_data.append(review)
      test_targets.append(y[i])
    return test_data,test_targets

In [19]:
train_data,train_tragets=text_preprocessing(x_train,y_train,traing=True)

100%|██████████| 43371/43371 [00:01<00:00, 37125.72it/s]


In [20]:
val_data,val_targets=text_preprocessing(x_val,y_val,validation=True)

100%|██████████| 10843/10843 [00:00<00:00, 39614.40it/s]


**#input string for model**

In [21]:
print(train_data[0])
f" traget value: {train_tragets[0]} : {le.inverse_transform([train_tragets[0]])[0]}"

TITLE : Rivals (1972) , DESCRIPTION : Scott Jacoby, as a boy with an unhealthy and pathological attachment to his divorced mother, becomes increasingly jealous of the new man in his mother's life (Robert Klein). After his mother remarries, his rage and misery overwhelm him and he plots to kill his stepfather.


' traget value: 8 :  drama '

**#model configuration parameters**

In [22]:
import transformers
device='cuda' if torch.cuda.is_available() else 'cpu'
max_len=256
train_batch_size=32
valid_batch_size=32
epochs=10
bert_path='bert-base-uncased'
model_path='models/bert_sentiment'
tokenizer=transformers.BertTokenizer.from_pretrained(bert_path,do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

**#data loader class**

In [23]:
class BERTDataset(Dataset):
  def __init__(self,review,target,tokenizer,max_len):
    self.review=review
    self.target=target
    self.tokenizer=tokenizer
    self.max_len=max_len

  def __len__(self):
    return len(self.review)

  def __getitem__(self,idx):
    review=str(self.review[idx])
    inputs=self.tokenizer.encode_plus(
        review,
        None,
        add_special_tokens=True,
        max_length=self.max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    ids=inputs['input_ids']
    mask=inputs['attention_mask']
    token_type_ids=inputs['token_type_ids']

    return{
        'ids':ids.squeeze(0),
        'mask':mask.squeeze(0),
        'token_type_ids':token_type_ids.squeeze(0),
        'targets':torch.tensor(self.target[idx],dtype=torch.long)
    }


**#data loader**

In [24]:
train_data=BERTDataset(train_data,train_tragets,tokenizer,max_len)
train_data_loader=DataLoader(train_data,batch_size=32,shuffle=True)

val_data=BERTDataset(val_data,val_targets,tokenizer,max_len)
val_data_loader=DataLoader(val_data,batch_size=32,shuffle=False)

In [25]:
f" total number of traing batch: {len(train_data_loader)}"

' total number of traing batch: 1356'

In [26]:
f"total  number of  validation batch: {len(val_data_loader)}"

'total  number of  validation batch: 339'

In [27]:
next(iter(train_data_loader))

{'ids': tensor([[ 101, 2516, 1024,  ...,    0,    0,    0],
         [ 101, 2516, 1024,  ...,    0,    0,    0],
         [ 101, 2516, 1024,  ...,    0,    0,    0],
         ...,
         [ 101, 2516, 1024,  ...,    0,    0,    0],
         [ 101, 2516, 1024,  ...,    0,    0,    0],
         [ 101, 2516, 1024,  ...,    0,    0,    0]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'targets': tensor([21,  5, 21,  7,  7,  7,  7,  0,  8,  5, 25,  5, 22,  7,  5,  8, 13,  5,
         21,  7, 13,  7, 19,  8,  7, 13,  7, 21,  7,  8,  7,  5])}

**#model fine tuning**

In [28]:
class BERTBaseUncased(nn.Module):
  def __init__(self):
    super(BERTBaseUncased,self).__init__()
    self.bert=transformers.BertModel.from_pretrained(bert_path)
    self.bert_drop=nn.Dropout(0.3)
    self.out=nn.Linear(768,27)
  def forward(self,ids,mask,token_type_ids):
    o2=self.bert(input_ids=ids,attention_mask=mask,token_type_ids=token_type_ids).last_hidden_state[:,0,:]
    bo=self.bert_drop(o2)
    output=self.out(bo)
    return output

In [29]:
model=BERTBaseUncased()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [30]:
model

BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

**#moving model to gpu**

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device='cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

In [32]:
torch.__version__

'2.5.1+cu121'

In [33]:
print(torch.version.cuda)

12.1


**#model hyperparameters**

In [34]:
loss = nn.CrossEntropyLoss()
num_train_steps = int(len(train_data) / train_batch_size * epochs)
optimizer = AdamW(model.parameters(),lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


**#EarlyStopping**

In [35]:

import numpy as np
import torch


class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_val_loss = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):
        # Check if validation loss is nan
        if np.isnan(val_loss):
            self.trace_func("Validation loss is NaN. Ignoring this epoch.")
            return

        if self.best_val_loss is None:
            self.best_val_loss = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss < self.best_val_loss - self.delta:
            # Significant improvement detected
            self.best_val_loss = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0  # Reset counter since improvement occurred
        else:
            # No significant improvement
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

**#traing step and validation**

In [38]:
def traing_step(train_data_loader,model,optimizer,device,scheduler,loss,val_data_loader,patience,epochs=epochs,):
  model.train()
  for epoch in range(epochs):
    train_loss=[]
    valid_loss=[]
    avg_train_loss=[]
    avg_valid_loss=[]
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    for batch in tqdm(train_data_loader):
       ###################
        # train the model #
        ###################
      ids=batch['ids'].to(device,dtype=torch.long)
      mask=batch['mask'].to(device,dtype=torch.long)
      token_type_ids=batch['token_type_ids'].to(device,dtype=torch.long)
      targets=batch['targets'].to(device,dtype=torch.long)
      # clear the gradients of all optimized variables
      optimizer.zero_grad()
       # forward pass: compute predicted outputs by passing inputs to the model
      outputs=model(ids=ids,mask=mask,token_type_ids=token_type_ids)
      outputs=outputs.squeeze(1)
      predicted_output=torch.argmax(outputs, dim=1)
      #print(predicted_output)
      # calculate the loss
      loss_value=loss(outputs,targets)
      # backward pass: compute gradient of the loss with respect to model parameters
      loss_value.backward()
      # perform a single optimization step (parameter update)
      optimizer.step()
      scheduler.step()
      train_loss.append(loss_value.item())
        ######################    
        # validate the model #
        ######################
    model.eval()
    with torch.no_grad():
      for batch in tqdm(val_data_loader):
        ids=batch['ids'].to(device,dtype=torch.long)
        mask=batch['mask'].to(device,dtype=torch.long)
        token_type_ids=batch['token_type_ids'].to(device,dtype=torch.long)
        targets=batch['targets'].to(device,dtype=torch.long)
        outputs=model(ids=ids,mask=mask,token_type_ids=token_type_ids)
        outputs=outputs.squeeze(1)
        loss_value=loss(outputs,targets)
        valid_loss.append(loss_value.item())

    # print training/validation statistics 
    # calculate average loss over an epoch
    train_losses=np.mean(train_loss)
    valid_losses=np.mean(valid_loss)
    avg_train_loss.append(train_losses)
    avg_valid_loss.append(valid_losses)

    epoch_len = len(str(epochs))
        
    print_msg = (f'[{epoch:>{epoch_len}}/{epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_losses:.5f} ' +
                     f'valid_loss: {valid_losses:.5f}')

    print(print_msg)
        
    # clear lists to track next epoch
    train_loss = []
    valid_loss = []
    # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
    early_stopping(valid_losses, model)
    
    if early_stopping.early_stop:
        print("Early stopping")
        break
 # load the last checkpoint with the best model
  model.load_state_dict(torch.load('checkpoint.pt', weights_only=True))

  return  model, avg_train_losses, avg_valid_losses

In [ ]:
# early stopping patience; how long to wait after last time validation loss improved.
patience=20
model, train_loss, valid_loss=traing_step(train_data_loader,model,optimizer,device,scheduler,loss,val_data_loader,patience,epochs=100)

**Visualizing the Loss and the Early Stopping Checkpoint**
From the plot we can see that the last Early Stopping Checkpoint was saved right before the model started to overfit.

In [ ]:

# visualize the loss as the network trained
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
plt.plot(range(1,len(valid_loss)+1),valid_loss,label='Validation Loss')

# find position of lowest validation loss
minposs = valid_loss.index(min(valid_loss))+1 
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0, 0.5) # consistent scale
plt.xlim(0, len(train_loss)+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot.png', bbox_inches='tight')

**#testing**

In [ ]:
test_df=pd.read_csv('/kaggle/input/genre-classification-20/Genre Classification Dataset/test_data.txt',sep=':::',header=None, names=['ID','TITLE','DESCRIPTION'])

In [ ]:
test_df.head()

In [ ]:
targets=pd.read_csv('/kaggle/input/genre-classification-20/Genre Classification Dataset/test_data_solution.txt',sep=':::',header=None, names=['ID','TITLE','GENRE','DESCRIPTION'])

In [ ]:
targets=targets['GENRE']

In [ ]:
targets.head()

**#data preprocessing and cleaning**

In [ ]:
test_df=test_df.drop(['ID'],axis=1)

In [ ]:
target=le.transform(targets)

In [ ]:
test_data,test_targets=text_preprocessing(test_df,target,Testing=True)

In [ ]:
test_data[0],f" label {test_targets[0]} : {le.inverse_transform([test_targets[0]])[0]}"

**#data loader**

In [ ]:
test_data=BERTDataset(test_data,test_targets,tokenizer,max_len)
test_data_loader=DataLoader(test_data,batch_size=32,shuffle=False)

In [ ]:
next(iter(test_data_loader))

**#testing step**

In [ ]:
def testing_step(test_data_loader,model,optimizer,device,scheduler):
  model.eval()
  test_outputs=[]
  model_outputs=[]

  with torch.no_grad():
    for batch in tqdm(test_data_loader):
      ids=batch['ids'].to(device,dtype=torch.long)
      mask=batch['mask'].to(device,dtype=torch.long)
      token_type_ids=batch['token_type_ids']
      targets=batch['targets']

      ids=ids.to(device,dtype=torch.long)
      mask=mask.to(device,dtype=torch.long)
      token_type_ids=token_type_ids.to(device,dtype=torch.long)
      targets=targets.to(device,dtype=torch.long)
      outputs=model(ids=ids,mask=mask,token_type_ids=token_type_ids)
      outputs=outputs.squeeze(1)
      predicted_output=torch.argmax(outputs, dim=1)

      test_outputs.extend(targets.cpu().detach().numpy())
      model_outputs.extend(predicted_output.cpu().detach().numpy())
  return test_outputs,model_outputs

In [ ]:
outputs,predicted_outputs=testing_step(test_data_loader,model,optimizer,device,scheduler)

In [ ]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(outputs,predicted_outputs)

In [ ]:
f"test accuracy score : {acc}"